In [1]:
from rdflib import Graph, Namespace, URIRef, RDF, RDFS
A = RDF['type']

In [2]:
filename = 'semantic-model.ttl'

In [3]:
g = Graph()
g.parse(filename, format = 'ttl')
BLDG = Namespace("https://BESTESTAir.urn#")
BR = Namespace("https://boptest-rules.urn#")
BRICK = Namespace("https://brickschema.org/schema/Brick#")
REF = Namespace("https://brickschema.org/schema/Brick/ref#")
g.bind('bldg',BLDG)

In [4]:
# adding points to AHU 
g.add((BLDG['ahu'], A, BRICK['AHU']))
for s, o in g.subject_objects(BR.pointOf):
    if o != BRICK['AHU']:
        raise ValueError(f'unexpected point of: {s} point of {o}')
    else:
        g.add((BLDG['ahu'], BRICK['hasPoint'], s))
        # print(f'{BLDG["ahu"]} has point {s}')

In [5]:
# loading brick ontology
g.parse('https://brickschema.org/schema/1.4.4/Brick.ttl', format='ttl')

<Graph identifier=Nfed183daf9a44bccb453cfb214d95abc (<class 'rdflib.graph.Graph'>)>

In [6]:
# adding feeds relationships
# getting list of AHU points
for vav in g.subjects(A, BRICK['VAV']):
    g.add((BLDG['ahu'], BRICK.feeds, vav))
    for zone in g.subjects(A, BRICK.Zone):
        if str(zone).split('_')[-1].lower() in vav:
            g.add((vav, BRICK.feeds, zone))
            print(f'{vav} feeds {zone}')
# TODO: Add points to Zones and VAVs
# Temperature and setpoints will be point of Zones 
# getting all points 
for point in g[:A / (RDFS.subClassOf * '+'): BRICK.Point]:
    if 'TZon' in str(point):
        for zone in g.subjects(A, BRICK.Zone):
            if str(zone).split('_')[-1] in point:
                g.add((zone, BRICK.hasPoint, point))
                print(f'Zone: {zone} has {point}')
    else:
        if ('hvac' in str(point)) and ('Zon' in str(point)):
            for vav in g.subjects(A, BRICK['VAV']):
                if str(vav).split('_')[-1].lower() in str(point).lower():
                    g.add((vav, BRICK.hasPoint, point))
                    print(f'VAV: {vav} has {point}')

https://BESTESTAir.urn#hvac_cor feeds https://BESTESTAir.urn#flo_Cor
https://BESTESTAir.urn#hvac_eas feeds https://BESTESTAir.urn#flo_Eas
https://BESTESTAir.urn#hvac_nor feeds https://BESTESTAir.urn#flo_Nor
https://BESTESTAir.urn#hvac_sou feeds https://BESTESTAir.urn#flo_Sou
https://BESTESTAir.urn#hvac_wes feeds https://BESTESTAir.urn#flo_Wes
VAV: https://BESTESTAir.urn#hvac_cor has https://BESTESTAir.urn#hvac_oveZonActCor_yDam
VAV: https://BESTESTAir.urn#hvac_eas has https://BESTESTAir.urn#hvac_oveZonActEas_yDam
VAV: https://BESTESTAir.urn#hvac_nor has https://BESTESTAir.urn#hvac_oveZonActNor_yDam
VAV: https://BESTESTAir.urn#hvac_sou has https://BESTESTAir.urn#hvac_oveZonActSou_yDam
VAV: https://BESTESTAir.urn#hvac_wes has https://BESTESTAir.urn#hvac_oveZonActWes_yDam
Zone: https://BESTESTAir.urn#flo_Cor has https://BESTESTAir.urn#hvac_oveZonSupCor_TZonCooSet
Zone: https://BESTESTAir.urn#flo_Eas has https://BESTESTAir.urn#hvac_oveZonSupEas_TZonCooSet
Zone: https://BESTESTAir.urn#flo_N

In [7]:
# checking for leftover points 
for point in g[:A / (RDFS.subClassOf * '+'): BRICK.Point]:
    if (None, BRICK.hasPoint, point) not in g:
        if 'chi' in str(point):
            g.add((BLDG['chi'], BRICK.hasPoint, point))
            print(f'chi: {BLDG["chi"]} has point {point}')
        elif 'heaPum' in str(point):
            g.add((BLDG['heaPum'], BRICK.hasPoint, point))
            print(f'heaPum: {BLDG["heaPum"]} has point {point}')
        else: 
            raise ValueError(f'{point} not linked')

chi: https://BESTESTAir.urn#chi has point https://BESTESTAir.urn#chi_reaPChi
chi: https://BESTESTAir.urn#chi has point https://BESTESTAir.urn#chi_reaPPumDis
heaPum: https://BESTESTAir.urn#heaPum has point https://BESTESTAir.urn#heaPum_reaPHeaPum
heaPum: https://BESTESTAir.urn#heaPum has point https://BESTESTAir.urn#heaPum_reaPPumDis
chi: https://BESTESTAir.urn#chi has point https://BESTESTAir.urn#chi_reaFloSup
heaPum: https://BESTESTAir.urn#heaPum has point https://BESTESTAir.urn#heaPum_reaFloSup
chi: https://BESTESTAir.urn#chi has point https://BESTESTAir.urn#chi_reaTSup
chi: https://BESTESTAir.urn#chi has point https://BESTESTAir.urn#chi_reaTRet


In [8]:
g.serialize('post-processed-model.ttl', format = 'ttl')

<Graph identifier=Nfed183daf9a44bccb453cfb214d95abc (<class 'rdflib.graph.Graph'>)>